Problem 1-Carseats Linear Regression

1(a) Fit a multiple linear regression model to predict Sales using Price, Urban, and US

In [ ]:
#데이터 관리 및 조작
import pandas as pd
df=pd.read_csv('/Users/lavi/MLDL_25_2_ASSIGNMENT/ASSIGNMENT_1/Data/Carseats.csv')

#Check Files for ensure
print(df.head())


   Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
0   9.50        138      73           11         276    120       Bad   42   
1  11.22        111      48           16         260     83      Good   65   
2  10.06        113      35           10         269     80    Medium   59   
3   7.40        117     100            4         466     97    Medium   55   
4   4.15        141      64            3         340    128       Bad   38   

   Education Urban   US  
0         17   Yes  Yes  
1         10   Yes  Yes  
2         12   Yes  Yes  
3         14   Yes  Yes  
4         13   Yes   No  


In [12]:
#변수 설정
Y=df['Sales']
X=df[['Price', 'Urban','US']]

#Check data for ensure
print(Y.head())
print(X.head())

0     9.50
1    11.22
2    10.06
3     7.40
4     4.15
Name: Sales, dtype: float64
   Price Urban   US
0    120   Yes  Yes
1     83   Yes  Yes
2     80   Yes  Yes
3     97   Yes  Yes
4    128   Yes   No


In [13]:
# Quali. var.(Urban, US) 인코딩
X_encoded = pd.get_dummies(X, drop_first=True)
print(X_encoded.head())

   Price  Urban_Yes  US_Yes
0    120       True    True
1     83       True    True
2     80       True    True
3     97       True    True
4    128       True   False


In [15]:
# 머신러닝 (Using Scikit-Learn)
from sklearn.linear_model import LinearRegression

lin_model=LinearRegression(fit_intercept=True)
lin_model.fit(X_encoded,Y)

r2=lin_model.score(X_encoded,Y)

# Output Check
print("Intercept: ",lin_model.intercept_)
print("Parameters: ", lin_model.coef_)
print("R_square: ", r2)


Intercept:  13.043468936764892
Parameters:  [-0.05445885 -0.02191615  1.2005727 ]
R_square:  0.23927539218405547


1(a) Report by pdf file.
Which contains the fitted coefficients (including the intercept) and the training R2 of the model.